In [1]:
import pandas as pd
import numpy as np 


In [2]:
df = pd.read_csv(r"C:\Users\UserOne\Desktop\Task IA\database.csv")
df.head(5)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c


In [3]:
#SETTING IT ON LOWER CASE
df["job_type"] = df["job_type"].str.lower()

#CHECKING THE UNIQUE VALUES IN JOB TYPE
print( "jobe type:", df["job_type" ].unique())

jobe type: ['full time employee' 'full time' 'full time, employee'
 'part time employee' nan 'full time temporary/contract/project'
 'full time , employee' 'full time, temporary/contract/project' 'employee'
 'part time' 'part time, employee' 'full time intern'
 'temporary/contract/project' 'full time / employee'
 'full time , temporary/contract/project'
 'part time, temporary/contract/project' 'full time/ employee'
 'per diem, employee' 'job type full time employee' 'per diem'
 'full time\xa0' 'part time intern' 'per diem employee'
 'part time/ temporary/contract/project'
 'part time temporary/contract/project' 'exempt'
 'part time , temporary/contract/project' 'full time\xa0 employee'
 'part time seasonal' 'part time , employee' 'job type employee'
 'job type full time temporary/contract/project' 'full time / > employee'
 'part time\xa0' 'per diem, temporary/contract/project'
 'full time / temporary/contract/project' 'part time, intern'
 'job type full time' 'part time / employee' 'jo

In [4]:
!pip install fuzzywuzzy

You should consider upgrading via the 'C:\Users\UserOne\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\UserOne\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
#CHECKING THE RATIOS THAT FUZZYWUZZY GIVE FOR JOB TYPES (FULL / PART)
matchesFT = fuzzywuzzy.process.extract("full time", df["job_type"].unique(), limit=15, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print("these are the matches of full time",":",matchesFT)

print("----------------------------------------------------")

matchesPT = fuzzywuzzy.process.extract("part time", df["job_type"].unique(), limit=15, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print("these are the matches of part time",":",matchesPT)


these are the matches of full time : [('full time', 100), ('full time\xa0', 100), ('full time intern', 72), ('full time employee', 67), ('full time, employee', 67), ('full time , employee', 67), ('full time / employee', 67), ('full time/ employee', 67), ('full time\xa0 employee', 67), ('full time / > employee', 67), ('job type full time', 67), ('part time', 56), ('part time\xa0', 56), ('job type full time employee', 50), ('part time employee', 44)]
----------------------------------------------------
these are the matches of part time : [('part time', 100), ('part time\xa0', 100), ('part time intern', 72), ('part time, intern', 72), ('part time employee', 67), ('part time, employee', 67), ('part time seasonal', 67), ('part time , employee', 67), ('part time / employee', 67), ('full time', 56), ('full time\xa0', 56), ('job type part time employee', 50), ('full time intern', 48), ('full time employee', 44), ('full time, employee', 44)]


In [8]:
#A FUNCTION THAT MATCH THE JOB_TYPE VALUES AND REPLACE IT BASED ON A MIN RATIO GIVEN AS A PARAMETER 

def replace_matches_in_column(df, column, string_to_match, min_ratio ):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=30, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
 #matches = fuzzywuzzy.process.extract(value, df["job_type"].unique(), limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [9]:
#INVOKING THE FUNCTION 
replace_matches_in_column(df, "job_type" , "full time", 60)
replace_matches_in_column(df, "job_type" , "part time ", 67)


All done!
All done!


In [10]:
df

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,full time,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,full time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,full time,DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,full time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,full time,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,United States of America,US,NaN,No,jobs.monster.com,This is a major premier Cincinnati based finan...,Assistant Vice President - Controller Job in C...,full time,"Cincinnati, OH",NaN,http://jobview.monster.com/Assistant-Vice-Pres...,"120,000.00 - 160,000.00 $ /yearbonus",NaN,a80bc8cc3a90c17eef418963803bc640
21996,United States of America,US,NaN,No,jobs.monster.com,Luxury homebuilder in Cincinnati seeking multi...,Accountant Job in Cincinnati,full time,"Cincinnati, OH 45236",Construction - Residential & Commercial/Office,http://jobview.monster.com/Accountant-Job-Cinc...,"45,000.00 - 60,000.00 $ /year",Manager (Manager/Supervisor of Staff),419a3714be2b30a10f628de207d041de
21997,United States of America,US,NaN,No,jobs.monster.com,RE: Adobe AEM- Client - Loca...,AEM/CQ developer Job in Chicago,full time,"Chicago, IL 60602",NaN,http://jobview.monster.com/AEM-CQ5-developer-J...,NaN,NaN,5a590350b73b2cec46b05750a208e345
21998,United States of America,US,NaN,No,jobs.monster.com,Jernberg Industries was established in 1937 an...,Electrician - Experienced Forging Electrician ...,full time,"Chicago, IL 60609","Jernberg Industries, Inc.",http://jobview.monster.com/Electrician-Experie...,25.00 - 28.00 $ /hour,Installation/Maintenance/Repair,40161cf61c283af9dc2b0a62947a5f1b


In [11]:
#SEEING THE LEFT JOB TYPES
print( "jobe type:", df["job_type" ].unique())

jobe type: ['full time' 'part time ' nan 'full time temporary/contract/project'
 'full time, temporary/contract/project' 'employee'
 'temporary/contract/project' 'full time , temporary/contract/project'
 'part time, temporary/contract/project' 'per diem, employee'
 'job type full time employee' 'per diem' 'per diem employee'
 'part time/ temporary/contract/project'
 'part time temporary/contract/project' 'exempt'
 'part time , temporary/contract/project' 'job type employee'
 'job type full time temporary/contract/project'
 'per diem, temporary/contract/project'
 'full time / temporary/contract/project' 'job type part time employee']


In [12]:
#MANUAL REPLACEMENT OF THE TYPES VULES USING LOGIC
#FULL JOB
df["job_type"]= df["job_type"].replace(['full time, temporary/contract/project', 'full time temporary/contract/project', 
                                        'full time , temporary/contract/project, ' , 'job type full time employee',
                                        'job type full time temporary/contract/project' ,'full time / temporary/contract/project', 
                                      'full time temporary/contract/project','full time , temporary/contract/project' ], "full time")
#PART JOB
df["job_type"]= df["job_type"].replace(['part time, temporary/contract/project','part time/ temporary/contract/project',
                                       'part time temporary/contract/project','part time , temporary/contract/project',
                                       'job type part time employee','part time '], "part time")
#OTHERS (UNSPICIFIED)
df["job_type"]= df["job_type"].replace(['exempt', 'temporary/contract/project',
                                        'per diem, employee', 'per diem','per diem employee', 'per diem, temporary/contract/project',
                                        'employee', 'job type employee']
                                       , "employee")


In [13]:
df

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,full time,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,full time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,full time,DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,full time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,full time,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,United States of America,US,NaN,No,jobs.monster.com,This is a major premier Cincinnati based finan...,Assistant Vice President - Controller Job in C...,full time,"Cincinnati, OH",NaN,http://jobview.monster.com/Assistant-Vice-Pres...,"120,000.00 - 160,000.00 $ /yearbonus",NaN,a80bc8cc3a90c17eef418963803bc640
21996,United States of America,US,NaN,No,jobs.monster.com,Luxury homebuilder in Cincinnati seeking multi...,Accountant Job in Cincinnati,full time,"Cincinnati, OH 45236",Construction - Residential & Commercial/Office,http://jobview.monster.com/Accountant-Job-Cinc...,"45,000.00 - 60,000.00 $ /year",Manager (Manager/Supervisor of Staff),419a3714be2b30a10f628de207d041de
21997,United States of America,US,NaN,No,jobs.monster.com,RE: Adobe AEM- Client - Loca...,AEM/CQ developer Job in Chicago,full time,"Chicago, IL 60602",NaN,http://jobview.monster.com/AEM-CQ5-developer-J...,NaN,NaN,5a590350b73b2cec46b05750a208e345
21998,United States of America,US,NaN,No,jobs.monster.com,Jernberg Industries was established in 1937 an...,Electrician - Experienced Forging Electrician ...,full time,"Chicago, IL 60609","Jernberg Industries, Inc.",http://jobview.monster.com/Electrician-Experie...,25.00 - 28.00 $ /hour,Installation/Maintenance/Repair,40161cf61c283af9dc2b0a62947a5f1b


In [14]:
print( "jobe type:", df["job_type" ].unique())

jobe type: ['full time' 'part time' nan 'employee']
